### import

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib import rcParams
rcParams['text.usetex'] = True
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Times New Roman']

import numpy as np
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report
from torchmetrics import AUROC
from torchmetrics.functional import accuracy
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc, confusion_matrix, accuracy_score
from sklearn import metrics
from torchmetrics import F1Score
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torch.nn.utils import clip_grad_norm_
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import torch.nn.functional as F
import torch
import torch.nn as nn
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from pytorch_lightning import Trainer, LightningModule
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl
from torchmetrics.classification import F1Score
from sklearn.metrics import precision_recall_fscore_support
import torch.nn.functional as F
from sklearn.metrics import classification_report
import torchmetrics
from torchmetrics import Metric
import torchmetrics.classification
import re
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pytorch_lightning.callbacks import LearningRateMonitor
from torchmetrics import Accuracy, MetricCollection, Precision, Recall, F1Score, Metric
from sklearn.metrics import confusion_matrix
from torchmetrics.classification import BinaryF1Score, BinaryConfusionMatrix
from torch import tensor
from typing import Optional
from pytorch_lightning.callbacks import Callback
import time
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import Trainer, seed_everything
import random
from torchmetrics import Accuracy, MetricCollection, Precision, Recall, ROC
from torch import tensor
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.profilers import PyTorchProfiler

In [ ]:
train_df_color = '#E9BFC0'          #red
val_df_color = '#C3D5E7'            #blue
test_df_color = '#F4D98F'           #yellow
threshold_color = '#DC96AE'         #pink

real_line_color = '#93c47d'         #blue
fake_line_color = '#E9BFC0'         #red
maybe_line_color = '#F4D98F'        #yellow

real_dot_color = '#56943a'          #green hard
fake_dot_color = '#8a292c'          #red red
maybe_dot_color = '#d69d00'         #yellow hard

logreg_line_color = '#7db364'       #green
logreg_fill_color = '#97c283'
logreg_dot_color = '#48802e'

binary_line_color = '#aeb1cf'       #purple
binary_dot_color = '#8a8db6'
binary_fill_color = '#bec0d8'       #3 tint lighter

multiclass_line_color = '#a2c9ef'   #blue
multiclass_fill_color = '#7a8c9f'
multiclass_dot_color = '#1e456b' 

colors = ['#d1e5f0', '#94c4f3', '#63a7e0', '#358ece', '#0a6ab7', '#084f8a'] #blues for CM
cmap = sns.color_palette(colors, as_cmap=True)

In [ ]:
def preprocess_text(text):
    text = text.lower()                                                         #lowercase   
    text = re.sub(r"@\w*", " ", text)                                           #tags
    text = re.sub(r"#|^\*|\*$|&quot;|&gt;|&lt;|&lt;3", " ", text)               #special characters
    text = text.replace("&amp;", " and ")                                       #&
    text = re.sub(r"ht+p+s?://\S*", " ", text)                                  #links
    text = re.sub(r"[^a-zA-Z0-9æøåÆØÅ\d\s:/£$%€,-]|(?<!\S)å(?!\S)", "", text)   #ascii
    text = re.sub(r",(?!\d)", "", text)                                         #comma
    text = re.sub(r"(\S)\.(\S)", r"\1. \2", text)                               #period
    text = re.sub(r"\(\)|\[\]|\{\}", " ", text)                                 #brackets
    text = re.sub(r"\s[b-gjz]\s", " ", text)                                    #single chars
    text = " ".join(text.split()).strip()                                       #spaces
    return text

In [ ]:
RANDOMSEED = 42
seed_everything(RANDOMSEED, workers=True) #for dataloaders
torch.set_float32_matmul_precision('medium')

In [ ]:
train_df = pd.read_csv('/home/thombras/files/NorLIAR_train_bin.csv')
val_df = pd.read_csv('/home/thombras/files/NorLIAR_val_bin.csv')
test_df = pd.read_csv('/home/thombras/files/NorLIAR_test_bin.csv')
PFO_test_df = pd.read_csv('/home/thombras/files/PFO.csv')
FND_test_df = pd.read_csv('/home/thombras/files/FND_gossip_df.csv')
LABEL_COLUMNS = train_df.columns.tolist()[6:]

In [ ]:
train_df['claim_no'] = train_df['claim_no'].apply(preprocess_text)
val_df['claim_no'] = val_df['claim_no'].apply(preprocess_text)
test_df['claim_no'] = test_df['claim_no'].apply(preprocess_text)
PFO_test_df['claim_no'] = PFO_test_df['claim_no'].apply(preprocess_text)
FND_test_df['claim_no'] = FND_test_df['claim_no'].apply(preprocess_text)
test_df.shape, PFO_test_df.shape, FND_test_df.shape, LABEL_COLUMNS

### norbert3

In [ ]:
BASE_MODEL = 'ltg/norbert3-base' #LTG = Language Technology Group (University of Oslo)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
norbert3 = AutoModel.from_pretrained(BASE_MODEL, trust_remote_code=True, output_hidden_states=True)

### encode

In [ ]:
BATCH_SIZE = 32
MAX_TOKEN_COUNT = 60
N_EPOCHS = 20
N_WORKERS = 2
INPUT_SIZE = norbert3.config.hidden_size 

class data_encode(Dataset):
    def __init__(self, data: pd.DataFrame, 
                tokenizer: AutoTokenizer, 
                MAX_TOKEN_COUNT: int = 128,
                labels=None): 
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = MAX_TOKEN_COUNT
    
    def __len__(self):                          
        return len(self.data)                   
    
    def __getitem__(self, index: int):  
        data_row = self.data.iloc[index]
        text = data_row.claim_no
        labels = data_row[LABEL_COLUMNS] 
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",                               #dynamic padding longest vs. max_length
            truncation=True,
            return_attention_mask=True, 
            return_tensors='pt', 
        )
        return dict(
            text=text,
            input_ids=encoding["input_ids"].flatten(),          #remove excess dim
            attention_mask=encoding["attention_mask"].flatten(),
            labels=torch.FloatTensor(labels))          

In [ ]:
train_dataset = data_encode(
    train_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

val_dataset = data_encode(
    val_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

test_dataset = data_encode(
    test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

PFO_test_dataset = data_encode(
    PFO_test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

FND_test_dataset = data_encode(
    FND_test_df,
    tokenizer,
    MAX_TOKEN_COUNT,
    LABEL_COLUMNS)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
PFO_test_loader = DataLoader(dataset=PFO_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)
FND_test_loader = DataLoader(dataset=FND_test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)   

### architecture

In [ ]:
from torch.utils.tensorboard import SummaryWriter

class NB3RegressorArch(pl.LightningModule):
    def __init__(self, base_model, lr: Optional[float] = None):
        super().__init__()
        self.save_hyperparameters(ignore=['base_model'])
        self.bert = base_model
        self.regressor_head = nn.Sequential(
            nn.Linear(768, 1))
        self.loss_func = nn.BCELoss()
        self.LR = lr
        self.acc = torchmetrics.classification.Accuracy(task='binary', average='weighted').cuda()
        self.prec = torchmetrics.classification.Precision(task='binary', average='weighted').cuda()
        self.rec = torchmetrics.classification.Recall(task='binary', average='weighted').cuda()
        self.f1 = torchmetrics.classification.F1Score(task='binary', average='weighted').cuda()
        self.auroc = torchmetrics.classification.AUROC(task='binary').cuda()
        self.confusion_matrix = torchmetrics.classification.ConfusionMatrix(task='binary').cuda()
        self.roc = torchmetrics.classification.ROC(task='binary').cuda()
        self.val_y_prob = []
        self.val_labels = []
        self.test_y_prob = []
        self.test_labels = []
        self.writer = SummaryWriter(log_dir="/home/light_logs/log_reg", filename_suffix='logreg_histo')
    
    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask=attention_mask)
        pooler = output.last_hidden_state[:, 0, :]  #CLS
        pooler_numpy = pooler.detach().cpu().numpy()
        self.writer.add_histogram('Pooler Output', pooler_numpy, self.global_step)
        logits = self.regressor_head(pooler)
        logits_numpy = logits.detach().cpu().numpy()
        self.writer.add_histogram('Logits', logits_numpy, self.global_step)
        y_prob = torch.sigmoid(logits)
        y_prob_numpy = y_prob.detach().cpu().numpy()
        self.writer.add_histogram('Probabilities', y_prob_numpy, self.global_step)
        return logits, y_prob 

    def _recurrent_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        _, y_prob = self.forward(input_ids, attention_mask)
        loss = self.loss_func(y_prob, labels)
        return loss, labels, y_prob
    
    def _calculate_metrics(self, batch, batch_idx):
        _, labels, y_prob = self._recurrent_step(batch, batch_idx)
        acc = self.acc(y_prob, labels)
        prec = self.prec(y_prob, labels)
        rec = self.rec(y_prob, labels)
        f1 = self.f1(y_prob, labels)
        return acc, prec, rec, f1, y_prob    

    def training_step(self, batch, batch_idx):
        loss, labels, _ = self._recurrent_step(batch, batch_idx)
        acc, prec, rec, f1, y_prob = self._calculate_metrics(batch, batch_idx)
        self.log_dict({'train_accuracy': acc, 'train_precision': prec, 'train_recall': rec, 'train_f1': f1},
                prog_bar=False, logger=True, on_step=True, on_epoch=True)
        self.log('train_loss', loss,  prog_bar=True, logger=True, on_step=True, on_epoch=True)
        return {'loss': loss, 'prediction': y_prob, 'labels': labels}

    def validation_step(self, batch, batch_idx):
        loss, labels, y_prob = self._recurrent_step(batch, batch_idx)
        acc, prec, rec, f1, y_prob = self._calculate_metrics(batch, batch_idx)
        self.val_y_prob.append(y_prob)
        self.val_labels.append(labels)
        self.log_dict({'val_accuracy': acc, 'val_f1': f1},
                prog_bar=False, logger=True, on_step=True, on_epoch=True)
        self.log('val_loss', loss,  prog_bar=True, logger=True, on_step=True, on_epoch=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        loss, labels, y_prob  = self._recurrent_step(batch, batch_idx)
        acc, prec, rec, f1,  y_prob = self._calculate_metrics(batch, batch_idx)
        self.test_y_prob.append(y_prob)
        self.test_labels.append(labels)
        self.log_dict({'test_accuracy': acc, 'test_precision': prec, 'test_recall': rec, 'test_f1': f1}, 
               prog_bar=False, logger=True, on_step=True, on_epoch=True)
        self.log('test_loss', loss, prog_bar=True, logger=True, on_step=True, on_epoch=True)
        return loss

    def on_train_end(self):
        self.writer.close()

    def on_validation_end(self):
        pass

    def on_test_end(self):
        test_y_prob_total = torch.cat(self.test_y_prob, dim=0)
        test_labels_total = torch.cat(self.test_labels, dim=0).long()
        print('test_y_prob_total\n', test_y_prob_total)
        print('test_labels_total\n', test_labels_total)
        
        #ROC
        self.roc.reset()
        self.roc.update(test_y_prob_total, test_labels_total)
        fpr, tpr, thresholds = self.roc.compute()
        fpr_cpu = fpr.cpu().numpy()
        tpr_cpu = tpr.cpu().numpy()
        best_threshold_index = np.argmax(tpr_cpu - fpr_cpu)
        plt.figure(figsize=(8, 5))
        plt.plot(fpr_cpu, tpr_cpu, color=logreg_line_color, lw=2, label='ROC curve')
        plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Reference Line')
        plt.plot(fpr_cpu[best_threshold_index], tpr_cpu[best_threshold_index], 'o', color=logreg_dot_color, label=f'Best Threshold: {thresholds[best_threshold_index]:.2f}')
        roc_treshold = thresholds[best_threshold_index]
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title('ROC for LR-NorBERT', fontsize=15)
        plt.legend(loc="lower right")
        print("FPR, TPR", fpr_cpu[best_threshold_index], tpr_cpu[best_threshold_index])
        print("Best threshold index", thresholds[best_threshold_index])
        plt.show()

        #AUROC
        test_y_prob_total_np = test_y_prob_total.cpu().numpy() #gpu to cpu + tensor to array
        test_labels_total_np = test_labels_total.cpu().numpy()
        fpr, tpr, thresholds = roc_curve(test_labels_total_np, test_y_prob_total_np)
        best_threshold = thresholds[np.argmax(tpr - fpr)]
        auroc = roc_auc_score(test_labels_total_np, test_y_prob_total_np)
        plt.figure(figsize=(8, 5))
        plt.plot(fpr, tpr, color=logreg_line_color, lw=2, label='ROC curve')
        plt.fill_between(fpr, tpr, color=logreg_fill_color, alpha=0.3)
        plt.text(0.5, 0.5, f'AUROC = {auroc:.2f}', fontsize=13, ha='right', va='top')

        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title('AUROC for LR-NorBERT', fontsize=15)
        plt.legend(loc="upper left")
        plt.show()
        print("AUROC value:", auroc)

        #CM
        print('thresh', roc_treshold)
        preds = (test_y_prob_total > roc_treshold).int()
        self.confusion_matrix.reset()
        test_end_cm = self.confusion_matrix(preds, test_labels_total)
        test_end_cm_np = test_end_cm.cpu().numpy()
        print("CM:", test_end_cm)
        
        labels = np.array([
            [f'True Negative\n{test_end_cm_np[0][0]}', f'False Positive\n{test_end_cm_np[0][1]}'],
            [f'False Negative\n{test_end_cm_np[1][0]}', f'True Positive\n{test_end_cm_np[1][1]}']])

        labels = np.asarray(labels).reshape(2, 2)
        plt.figure(figsize=(8, 5))
        sns.heatmap(test_end_cm_np, annot=labels, cmap=cmap, fmt="", annot_kws={"size": 13})
        plt.ylabel("True Class", fontsize=12)
        plt.xlabel("Predicted Class", fontsize=12)
        plt.title("CM for LR-NorBERT", fontsize=15)
        plt.show()

        test_y_prob_total_cpu = test_y_prob_total.cpu()
        test_labels_total_cpu = test_labels_total.cpu()
        print('best_threshold',best_threshold)
        predictions = (test_y_prob_total_cpu > best_threshold).int()
        target_names = ['Fake', 'Real']
        print(classification_report(test_labels_total_cpu, predictions, target_names=target_names))
        
        self.test_y_prob = [] #reset the arrays before new test dataloader
        self.test_labels = []
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.regressor_head.parameters(), lr=self.LR)
        return optimizer

class TrainingTime(Callback):
    def on_train_start(self, trainer, NB3RegressorArch):
        self.start = time.time()
    
    def on_train_end(self, trainer, NB3RegressorArch):
        self.end = time.time()
        total_minutes = (self.end-self.start)
        print(f'\nTraining finished: {total_minutes:.2f} seconds\n')

### fine-tune

In [ ]:
MONITOR_METRIC = 'val_loss'
MON_MODE = 'min'


#hyperparameters
TUNER_LR = 2.09e-6
LR = TUNER_LR      
DR = 0.2
DEVICE = 'gpu'
PATIENCE = 1
TOP_K = 1
DEV_RUN = 0

TB_LOGGER = TensorBoardLogger('/home/light_logs',
            name='lr')

checkpoint_callback = ModelCheckpoint(
  dirpath = '/home/checkpoints',
  filename = 'best_lr',
  save_top_k = TOP_K, 
  verbose = True,
  monitor = MONITOR_METRIC,   
  mode = MON_MODE)

LR_MONITOR = LearningRateMonitor(logging_interval='epoch')

early_stopping_callback = EarlyStopping(
    monitor=MONITOR_METRIC, 
    patience=PATIENCE) 

nb3 = norbert3  #base model
reg_model = NB3RegressorArch(
    base_model=nb3,
    lr=LR)

trainer = pl.Trainer(
    fast_dev_run=DEV_RUN,
    logger=TB_LOGGER,
    min_epochs=1,
    max_epochs=N_EPOCHS,
    accelerator = DEVICE,
    callbacks=[checkpoint_callback, early_stopping_callback, 
               TrainingTime(), LR_MONITOR]
)


print("\nFast_dev_run =", trainer.fast_dev_run)
print("Max epoch:", N_EPOCHS)
print("Batch size:", BATCH_SIZE)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.fit(reg_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

### evaluate and test

In [ ]:
trainer.validate(reg_model, dataloaders=val_loader)

In [ ]:
m_version = '/home/checkpoints/best_lr.ckpt'
trainer.test(ckpt_path=m_version, dataloaders=test_loader)
#test_loader
#PFO_test_loader
#FND_test_loader